# Huggingface
Hugging provide two ways to invoke LLM
1. By Api token 
2. local model


### install Libs

In [ ]:
!pip install langchain huggingface_hub transformers sentence_transformers accelerate bitsandbytes

In [ ]:
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
### create pormpt template
question = "Where is the capital of China?"
template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
repo_id ="google/flan-t5-base"

In [ ]:
llm = HuggingFaceHub(repo_id=repo_id)
llm_chain = LLMChain(prompt=prompt, llm=llm,llm_kwargs={"temperature":0,"max_length":512})
print(llm_chain.run(question))


## Create RAG pipeline

In [ ]:
! pip install pypdf faiss-cpu

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/Eng_11_Qiulu.pdf")
pages = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages[:4])

In [ ]:
# pip install --upgrade langchain_community
#! pip uninstall InstructorEmbedding

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS



In [ ]:
# 不需要传递 api_key，HuggingFaceInstructEmbeddings 会自动从环境变量读取 API 密钥
embedding = HuggingFaceInferenceAPIEmbeddings(api_key = HUGGINGFACEHUB_API_TOKEN,model_name="sentence-transformers/all-MiniLM-L6-v2")

# 假设 docs 已经准备好了文档列表
db = FAISS.from_documents(docs, embedding)

# 执行相似度搜索
query = "What's the person name?"
result_slmii = db.similarity_search(query, k=3)

In [ ]:
source_knowledge = "\n".join([x.page_content for x in result_slmii])

In [ ]:
augmented_prompt = """Using the contexts below, answer the query.
contexts:
{source_knowledge}
query:
{query}
"""

In [ ]:
prompt = PromptTemplate(template=augmented_prompt,input_variables=["source_knowledge","query"])

llm_chain = LLMChain(prompt=prompt, llm=llm,llm_kwargs={"temperature":0,"max_length":1024})


In [ ]:
print(llm_chain.run({"source_knowledge": source_knowledge,"query":query}))

In [ ]:
argumented_prompt_2=f"""Using the contexts below, anser the query.
contexts:
{source_knowledge}
query:
{query}
"""

In [ ]:
print(argumented_prompt_2)